In [ ]:
import awkward as ak
import numpy as np
import hist
import matplotlib as mpl
from coffea.nanoevents import NanoEventsFactory, DelphesSchema
DelphesSchema.mixins["FatJet"] = "Jet"

In [ ]:
root_fname = "events.root"
Events = NanoEventsFactory.from_root(
    file=root_fname,
    treepath="Delphes",
    schemaclass=DelphesSchema,
).events()

In [ ]:
# require two jets
events = Events
mask = ak.num(events.FatJet)>=2
events = events[mask]

# Dijet
events["Dijet"] = events.FatJet[:,0]+events.FatJet[:,1]

# transverse mass calculation
def ET(vec):
    return np.sqrt(vec.px**2+vec.py**2+vec.mass**2)
E1 = ET(events.Dijet)
E2 = events.MissingET.MET
MTsq = (E1+E2)**2-(events.Dijet.px+events.MissingET.px)**2-(events.Dijet.py+events.MissingET.py)
events["MT"] = np.sqrt(MTsq, where=MTsq>=0)
    
# 4-vectors for dijet
events["Jet_pt"] = events.Dijet.pt
events["Jet_eta"] = events.Dijet.eta
events["Jet_phi"] = events.Dijet.phi
events["Jet_mass"] = events.Dijet.mass
# Other quantites to be plotted
events["MET"] = events.MissingET.MET
events["DeltaEta"] = events.FatJet.DeltaEta
events["DeltaPhi"] = events.FatJet.DeltaPhi

## For plotting individually for jet1 and jet2
leading_jets = events.FatJet[:, :2]

jet1_mask = leading_jets[:, 0].pt >= leading_jets[:, 1].pt
jet2_mask = ~jet1_mask

events["Jet1"] = ak.where(jet1_mask, leading_jets[:, 0], leading_jets[:, 1])
events["Jet2"] = ak.where(jet2_mask, leading_jets[:, 0], leading_jets[:, 1])

# 4-vectors for jet1 and jet2

events["Jet1_pt"] = events["Jet1"].pt
events["Jet2_pt"] = events["Jet2"].pt
events["Jet1_eta"] = events["Jet1"].eta
events["Jet2_eta"] = events["Jet2"].eta
events["Jet1_phi"] = events["Jet1"].phi
events["Jet2_phi"] = events["Jet2"].phi
events["Jet1_mass"] = events["Jet1"].mass
events["Jet2_mass"] = events["Jet2"].mass
events["DeltaEta_MET_Jet1"] = np.abs(events.MissingET.eta - events["Jet1_eta"])
events["DeltaEta_MET_Jet2"] = np.abs(events.MissingET.eta - events["Jet2_eta"])
events["DeltaPhi_MET_Jet1"] = np.abs(events.MissingET.phi - events["Jet1_phi"])
events["DeltaPhi_MET_Jet2"] = np.abs(events.MissingET.phi - events["Jet2_phi"])

In [ ]:
dark_hadron_ids = [4900111, 4900113, 4900211, 4900213]
stable_particle_ids = [51, 52, 53]
pid = events.Particle["PID"]
d1 = events.Particle["D1"]

# Boolean array of whether a particle is dark
is_dark = ak.zeros_like(pid)
for dhid in dark_hadron_ids:
    is_dark = is_dark | (np.abs(pid)==dhid)
is_dark = is_dark==1
    
# PIDs of dark daughter
dark_daughter = pid[d1[is_dark]] 
is_dark_daughter = ak.zeros_like(dark_daughter)

for dsid in stable_particle_ids:
    is_dark_daughter = is_dark_daughter | (np.abs(dark_daughter)==dsid)
    
stability = [None]*len(events)
for i in range(len(pid)): 
    if ak.sum(is_dark[i])!=0:
        stability[i] = ak.sum(is_dark_daughter[i])/ak.sum(is_dark[i])
        
#stability = [ak.sum(is_dark_daughter[i])/ak.sum(is_dark[i]) for i in range(len(pid)) if ak.sum(is_dark[i])!=0]

# Add the invisible fraction to the events
events["stable_invisible_fraction"] = stability
    
# store modified events array
Events = events

In [ ]:
## Histograms 

# helper functions to fill histograms

def get_values(var,Events):
    return ak.flatten(Events[var],axis=None)

def fill_hist(var,nbins,bmin,bmax,label,Events):
    h = (
        hist.Hist.new
        .Reg(nbins, bmin, bmax, label=label)
        .Double()
    )
    h.fill(get_values(var,Events),weight=0.5)
    return h

# plot parameters
hists_mt = fill_hist("MT",25,0,1500,r"$m_{\text{T}}$ [GeV]",Events)
hists_pt = fill_hist("Jet_pt",50,0,1000,r"$p_{\text{T}}$ [GeV]",Events)
hists_pt1 = fill_hist("Jet1_pt",50,0,1000,r"$Jet_1\_p_{\text{T}}$ [GeV]",Events)
hists_pt2 = fill_hist("Jet2_pt",50,0,1000,r"$Jet_2\_p_{\text{T}}$ [GeV]",Events)
hists_met = fill_hist("MET",50,0,1000,r"$p_{\text{T},miss}$ [GeV]",Events)
hists_eta = fill_hist("Jet_eta",50,-10,10,r"$Jet_{\eta}$ [GeV]",Events)
hists_eta1 = fill_hist("Jet1_eta",50,-6,6,r"$\eta_{Jet1}$",Events)
hists_eta2 = fill_hist("Jet2_eta",50,-6,6,r"$\eta_{Jet2}$",Events)
hists_deltaeta = fill_hist("DeltaEta",35,0,2.0,r"$\Delta\eta$",Events)
hists_phi = fill_hist("Jet_phi",25,-4,4,r"$Jet_{\phi}$",Events)
hists_phi1 = fill_hist("Jet1_phi",25,-4,4,r"$\phi_{Jet1}$",Events)
hists_phi2 = fill_hist("Jet2_phi",25,-4,4,r"$\phi_{Jet2}$",Events)
hists_deltaphi = fill_hist("DeltaPhi",20,0,1.1,r"$\Delta\Phi$",Events)
hists_mass = fill_hist("Jet_mass",50,0,2300,r"$Jet_{mass}$ [GeV]",Events)
hists_mass1 = fill_hist("Jet1_mass",50,0,250,r"$Jet1_{mass}$ [GeV]",Events)
hists_mass2 = fill_hist("Jet2_mass",50,0,250,r"$Jet2_{mass}$ [GeV]",Events)
hists_deltaeta1 = fill_hist("DeltaEta_MET_Jet1",25,0,12,r"$\Delta\eta\_P_{t,miss}\_Jet_1$",Events)
hists_deltaeta2 = fill_hist("DeltaEta_MET_Jet2",25,0,12,r"$\Delta\eta\_P_{t,miss}\_Jet_2$",Events)
hists_deltaphi1 = fill_hist("DeltaPhi_MET_Jet1",25,0,7,r"$\Delta\phi\_P_{t,miss}\_Jet_1$",Events)
hists_deltaphi2 = fill_hist("DeltaPhi_MET_Jet2",25,0,7,r"$\Delta\phi\_P_{t,miss}\_Jet_2$",Events)
hists_if = fill_hist("stable_invisible_fraction",25,0,1,r"stable_invisible_fraction",Events)

hists = [hists_mt, hists_pt, hists_pt1, hists_pt2, hists_met, hists_eta, hists_eta1, hists_eta2, hists_deltaeta, 
         hists_phi, hists_phi1, hists_phi2, hists_deltaphi, hists_mass, hists_mass1, hists_mass2, hists_deltaeta1, hists_deltaeta2,
        hists_deltaphi1, hists_deltaphi2, hists_if]

In [ ]:
# Saving the histograms

import pickle

with open(f"../Save_hist/{name}.pkl", "wb") as out:
    for item in hists:            
        pickle.dump(item, out)

In [ ]:
hnames = ["Jet_mt", "Jet_pt", "Jet1_pt", "Jet2_pt", "MET", "Jet_eta", "Jet1_eta", "Jet2_eta", "DeltaEta", 
          "Jet_phi", "Jet1_phi", "Jet2_phi", "DeltaPhi", "Jet_mass", "Jet1_mass", "Jet2_mass", 
          "DeltaEta_MET_Jet1", "DeltaEta_MET_Jet2", "DeltaPhi_MET_Jet1", "DeltaPhi_MET_Jet2", "stable_invisible_fraction"]

In [ ]:
hists = {}      # Contains the lists of histos for all models

for sample in samples:
    path = f'../models/{sample["model"]}'
    file=f'{path}/Hists.pkl'
    hists_model = []                  # Contains all the histos for 1 model
    with open(file, "rb") as inp:
        while True:            
            try:
                hists_model.append(pickle.load(inp))            
            except EOFError:
                break
                
    hists[sample["name"]] = hists_model
    
# helper to make a plot
def make_plot(hname,hists):                         # hists is a dict containing 
    fig, ax = plt.subplots(figsize=(8,6))
    for l,h in hists.items():                       # h is a list of hist objects
        hep.histplot(h[hname],density=True,ax=ax,label=l)
    ax.set_xlim(h.axes[0].edges[0],h.axes[0].edges[-1])
    ax.set_yscale("log")
    ax.set_ylabel("Arbitury unit")
    ax.legend(framealpha=0.5)
    plt.savefig('All_plots/{}.pdf'.format(hname),bbox_inches='tight')
    
def make_all_plots(hnames):
    for hname in hnames:
        make_plot(hname, hists)